# Feature Selection - Part 1
## Understanding Feature Selection in Machine Learning

In this notebook, we'll explore Feature Selection techniques and their implementation in Python.

## What is Feature Selection?

Feature Selection is the process of automatically selecting the most relevant features (variables, predictors) for your model. It's a crucial step in the machine learning pipeline that:

1. Identifies the most important features
2. Removes irrelevant or redundant features
3. Reduces the dimensionality of your dataset
4. Improves model performance

## Why do we need Feature Selection?

Feature Selection is important for several reasons:

1. **Reduces Overfitting**: Fewer redundant features means less opportunity to make decisions based on noise
2. **Improves Accuracy**: Less misleading data means model accuracy improves
3. **Reduces Training Time**: Fewer features mean faster training
4. **Enhanced Generalization**: Reduces variance and helps avoid overfitting
5. **Better Interpretability**: Fewer features make it easier to explain the model's decisions

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import VarianceThreshold, f_classif, chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [ ]:
# Load sample dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print("Dataset shape:", X.shape)
print("\nFeature names:\n", data.feature_names)

## Types of Feature Selection

There are three main types of feature selection methods:

1. **Filter Methods**: Use statistical measures to score features
2. **Wrapper Methods**: Use model performance to evaluate feature subsets
3. **Embedded Methods**: Perform feature selection during model training

In this notebook, we focus on Filter Methods.

## Filter-based Feature Selection

Filter methods use statistical measures to score the correlation or dependence between input variables that can be filtered to choose the most relevant features. These methods are:
- Computationally fast
- Independent of the learning algorithm
- Usually univariate

Let's explore different filter methods:

### 1. Duplicate Features

First, let's check for duplicate features in our dataset.

In [ ]:
# Check for duplicate features
def find_duplicates(df):
    duplicates = {}
    columns = df.columns
    
    for i in range(len(columns)):
        for j in range(i+1, len(columns)):
            if df[columns[i]].equals(df[columns[j]]):
                duplicates[columns[j]] = columns[i]
    
    return duplicates

duplicates = find_duplicates(X)
print("Duplicate features:", duplicates if duplicates else "None found")

### 2. Variance Threshold

Variance Threshold is a simple baseline approach to feature selection. It removes features whose variance doesn't meet a certain threshold.

In [ ]:
# Apply Variance Threshold
selector = VarianceThreshold(threshold=0.01)
X_scaled = StandardScaler().fit_transform(X)
X_var_selected = selector.fit_transform(X_scaled)

# Get selected features
selected_features = X.columns[selector.get_support()].tolist()

print("Original number of features:", X.shape[1])
print("Number of selected features:", len(selected_features))
print("\nSelected features:\n", selected_features)

### 3. Correlation Analysis

Correlation analysis helps identify redundant features by measuring the linear correlation between features.

In [ ]:
# Calculate correlation matrix
correlation_matrix = X.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Find highly correlated features
def find_high_correlations(correlation_matrix, threshold=0.9):
    high_corr = np.where(np.abs(correlation_matrix) > threshold)
    high_corr = [(correlation_matrix.index[x], correlation_matrix.columns[y], correlation_matrix.iloc[x, y]) 
                 for x, y in zip(*high_corr) if x != y and x < y]
    return pd.DataFrame(high_corr, columns=['Feature 1', 'Feature 2', 'Correlation'])

high_correlations = find_high_correlations(correlation_matrix)
print("\nHighly correlated features (>0.9):\n")
print(high_correlations)

### 4. ANOVA (Analysis of Variance)

ANOVA is used to determine the statistical significance of features for classification problems.

In [ ]:
# Apply ANOVA F-test
f_scores, p_values = f_classif(X_scaled, y)

# Create DataFrame with feature scores
feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'F Score': f_scores,
    'P Value': p_values
})

# Sort by F-score
feature_scores = feature_scores.sort_values('F Score', ascending=False)
print("Top 10 features by ANOVA F-score:\n")
print(feature_scores.head(10))

# Plot F-scores
plt.figure(figsize=(12, 6))
plt.bar(range(len(f_scores)), f_scores)
plt.xticks(range(len(f_scores)), X.columns, rotation=90)
plt.title('ANOVA F-scores for each feature')
plt.tight_layout()
plt.show()

### 5. Chi-Square Test

Chi-Square test is another statistical test for feature selection, particularly useful for categorical features.

In [ ]:
# Apply Chi-Square test
# Note: Chi-square requires non-negative values, so we use MinMaxScaler
X_scaled_positive = MinMaxScaler().fit_transform(X)
chi_scores, chi_p_values = chi2(X_scaled_positive, y)

# Create DataFrame with feature scores
chi_feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'Chi-Square Score': chi_scores,
    'P Value': chi_p_values
})

# Sort by Chi-Square score
chi_feature_scores = chi_feature_scores.sort_values('Chi-Square Score', ascending=False)
print("Top 10 features by Chi-Square score:\n")
print(chi_feature_scores.head(10))

## Advantages and Disadvantages of Filter Methods

### Advantages:
1. **Fast**: Computationally efficient
2. **Scalable**: Can handle large datasets
3. **Independent**: Works independently of the learning algorithm
4. **Univariate**: Each feature is considered independently

### Disadvantages:
1. **Ignores Feature Dependencies**: Doesn't account for feature interactions
2. **Ignores Learning Algorithm**: May select features that aren't optimal for the specific model
3. **Redundancy**: May select redundant features (unless explicitly checked)
4. **Arbitrary Thresholds**: Often requires manual threshold setting